<a href="https://colab.research.google.com/github/akiran703/MultiInputPatientApproach/blob/master/finalpredictionmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import Image
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow.keras as tf
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten,Concatenate, Dropout
from tensorflow.keras.utils import to_categorical
import os
import pandas as pd
import csv
import cv2
import random
import glob
from IPython.core.debugger import Tracer


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth 
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
download_CT_COVID = drive.CreateFile({'id':'1bDI0Lokr1Do1adDq6kY2I6NIedaPGDbS'})
download_CT_COVID.GetContentFile('clustercovid.zip')

In [ ]:
!unzip /content/clustercovid.zip -d /content/Data

Archive:  /content/clustercovid.zip
   creating: /content/Data/clustercovid/
  inflating: /content/Data/clustercovid/.DS_Store  
  inflating: /content/Data/__MACOSX/clustercovid/._.DS_Store  
   creating: /content/Data/clustercovid/coviddata/
  inflating: /content/Data/clustercovid/firstcovid.py  
  inflating: /content/Data/__MACOSX/clustercovid/._firstcovid.py  
   creating: /content/Data/clustercovid/coviddata/CT_COVID/
  inflating: /content/Data/__MACOSX/clustercovid/coviddata/._CT_COVID  
  inflating: /content/Data/clustercovid/coviddata/.DS_Store  
  inflating: /content/Data/__MACOSX/clustercovid/coviddata/._.DS_Store  
   creating: /content/Data/clustercovid/coviddata/CT_NonCOVID/
  inflating: /content/Data/__MACOSX/clustercovid/coviddata/._CT_NonCOVID  
  inflating: /content/Data/clustercovid/coviddata/CT_COVID/2020.03.01.20029769-p21-73_1%1.png  
  inflating: /content/Data/__MACOSX/clustercovid/coviddata/CT_COVID/._2020.03.01.20029769-p21-73_1%1.png  
  inflating: /content/Data

In [ ]:
from google.colab import files
file = files.upload()

dfcovidtest = pd.read_csv('update_covid_test.csv')
patientidcovidtest = dfcovidtest["Patient ID"].value_counts().keys().tolist()
countcovidtest = dfcovidtest["Patient ID"].value_counts().tolist()


dfcovidtrain = pd.read_csv('update_covid_train.csv')
patientidcovidtrain = dfcovidtrain["Patient ID"].value_counts().keys().tolist()
countcovidtrain = dfcovidtrain["Patient ID"].value_counts().tolist()


dfcovidval = pd.read_csv('update_covid_val.csv')
patientidcovidval = dfcovidval["Patient ID"].value_counts().keys().tolist()
countcovidval = dfcovidval["Patient ID"].value_counts().tolist()

Saving update_covid_test.csv to update_covid_test.csv
Saving update_covid_train.csv to update_covid_train.csv
Saving update_covid_val.csv to update_covid_val.csv


In [ ]:

from google.colab import files
file = files.upload()

dfnoncovidtest = pd.read_csv('update_noncovid_test.csv')
patientidnoncovidtest = dfnoncovidtest["patient id"].value_counts().keys().tolist()
countnoncovidtest = dfnoncovidtest["patient id"].value_counts().tolist()



dfnoncovidtrain = pd.read_csv('update_noncovid_train.csv')
patientidnoncovidtrain = dfnoncovidtrain["patient id"].value_counts().keys().tolist()
countnoncovidtrain = dfnoncovidtrain["patient id"].value_counts().tolist()



dfnoncovidval = pd.read_csv('update_noncovid_val.csv')
patientidnoncovidval = dfnoncovidval["patient id"].value_counts().keys().tolist()
countnoncovidval = dfnoncovidval["patient id"].value_counts().tolist()

Saving update_noncovid_test.csv to update_noncovid_test.csv
Saving update_noncovid_train.csv to update_noncovid_train.csv
Saving update_noncovid_val.csv to update_noncovid_val.csv


In [ ]:
covidtest = []
covidtrain = []
covidval = []

covidtestlabel = []
covidtrainlabel = []
covidvallabel = []

covidtestid = []
covidtrainid = []
covidvalid = []

noncovidtest = []
noncovidtrain = []
noncovidval = []

noncovidtestlabel = []
noncovidtrainlabel = []
noncovidvallabel = []


noncovidtestid = []
noncovidtrainid = []
noncovidvalid = []


In [ ]:
desired_size=200
inputparameter = 1

In [ ]:
def myFunc(name):
        img = cv2.imread(name)
        old_size = img.shape[:2]
        ratio = float(desired_size) / max(old_size)
        new_size = tuple([int(x * ratio) for x in old_size])
        img = cv2.resize(img, (new_size[1], new_size[0]))
        delta_w = desired_size - new_size[1]
        delta_h = desired_size - new_size[0]
        top, bottom = delta_h // 2, delta_h - (delta_h // 2)
        left, right = delta_w // 2, delta_w - (delta_w // 2)
        color = [0, 0, 0]
        imgreszie = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
        normresize = cv2.normalize(imgreszie, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        covidtest.append(normresize)
        covidtestlabel.append(1)



def trainmyFunc(name):
        img = cv2.imread(name)
        old_size = img.shape[:2]
        ratio = float(desired_size) / max(old_size)
        new_size = tuple([int(x * ratio) for x in old_size])
        img = cv2.resize(img, (new_size[1], new_size[0]))
        delta_w = desired_size - new_size[1]
        delta_h = desired_size - new_size[0]
        top, bottom = delta_h // 2, delta_h - (delta_h // 2)
        left, right = delta_w // 2, delta_w - (delta_w // 2)
        color = [0, 0, 0]
        imgreszie = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
        normresize = cv2.normalize(imgreszie, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        covidtrain.append(normresize)
        covidtrainlabel.append(1)



def valmyFunc(name):
        img = cv2.imread(name)
        old_size = img.shape[:2]
        ratio = float(desired_size) / max(old_size)
        new_size = tuple([int(x * ratio) for x in old_size])
        img = cv2.resize(img, (new_size[1], new_size[0]))
        delta_w = desired_size - new_size[1]
        delta_h = desired_size - new_size[0]
        top, bottom = delta_h // 2, delta_h - (delta_h // 2)
        left, right = delta_w // 2, delta_w - (delta_w // 2)
        color = [0, 0, 0]
        imgreszie = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
        normresize = cv2.normalize(imgreszie, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        covidval.append(normresize)
        covidvallabel.append(1)



def seekvalue(y):
    findValue = dfcovidtest.loc[dfcovidtest['Patient ID'] == y]
    for i, row in findValue.iterrows():
        image_value = f"{row['File name']}"
        f1 = f'/content/Data/clustercovid/coviddata/CT_COVID/{image_value}'
        covidtestid.append(y)
        myFunc(f1)


def trainseekvalue(y):
    findValue = dfcovidtrain.loc[dfcovidtrain['Patient ID'] == y]
    for i, row in findValue.iterrows():
        image_value = f"{row['File name']}"
        f1 = f'/content/Data/clustercovid/coviddata/CT_COVID/{image_value}'
        covidtrainid.append(y)
        trainmyFunc(f1)

def valseekvalue(y):
    findValue = dfcovidval.loc[dfcovidval['Patient ID'] == y]
    for i, row in findValue.iterrows():
        image_value = f"{row['File name']}"
        f1 = f'/content/Data/clustercovid/coviddata/CT_COVID/{image_value}'
        covidvalid.append(y)
        valmyFunc(f1)



for i in range(len(patientidcovidtest)):
    x = seekvalue(patientidcovidtest[i])


for i in range(len(patientidcovidtrain)):
    x = trainseekvalue(patientidcovidtrain[i])

for i in range(len(patientidcovidval)):
    x = valseekvalue(patientidcovidval[i])

In [ ]:
def nonmyFunc(name):
        img = cv2.imread(name)
        old_size = img.shape[:2]
        ratio = float(desired_size) / max(old_size)
        new_size = tuple([int(x * ratio) for x in old_size])
        img = cv2.resize(img, (new_size[1], new_size[0]))
        delta_w = desired_size - new_size[1]
        delta_h = desired_size - new_size[0]
        top, bottom = delta_h // 2, delta_h - (delta_h // 2)
        left, right = delta_w // 2, delta_w - (delta_w // 2)
        color = [0, 0, 0]
        imgreszie = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
        normresize = cv2.normalize(imgreszie, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        noncovidtest.append(normresize)
        noncovidtestlabel.append(0)


def nontrainmyFunc(name):
        img = cv2.imread(name)
        old_size = img.shape[:2]
        ratio = float(desired_size) / max(old_size)
        new_size = tuple([int(x * ratio) for x in old_size])
        img = cv2.resize(img, (new_size[1], new_size[0]))
        delta_w = desired_size - new_size[1]
        delta_h = desired_size - new_size[0]
        top, bottom = delta_h // 2, delta_h - (delta_h // 2)
        left, right = delta_w // 2, delta_w - (delta_w // 2)
        color = [0, 0, 0]
        imgreszie = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
        normresize = cv2.normalize(imgreszie, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        noncovidtrain.append(normresize)
        noncovidtrainlabel.append(0)


def nonvalmyFunc(name):
        img = cv2.imread(name)
        old_size = img.shape[:2]
        ratio = float(desired_size) / max(old_size)
        new_size = tuple([int(x * ratio) for x in old_size])
        img = cv2.resize(img, (new_size[1], new_size[0]))
        delta_w = desired_size - new_size[1]
        delta_h = desired_size - new_size[0]
        top, bottom = delta_h // 2, delta_h - (delta_h // 2)
        left, right = delta_w // 2, delta_w - (delta_w // 2)
        color = [0, 0, 0]
        imgreszie = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
        normresize = cv2.normalize(imgreszie, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        noncovidval.append(normresize)
        noncovidvallabel.append(0)





def nonseekvalue(y):
    findValue = dfnoncovidtest.loc[dfnoncovidtest['patient id'] == y]
    for i, row in findValue.iterrows():
        image_value = f"{row['image name']}"
        f1 = f'/content/Data/clustercovid/coviddata/CT_NonCOVID/{image_value}'
        noncovidtestid.append(y)
        nonmyFunc(f1)


def nontrainseekvalue(y):
    findValue = dfnoncovidtrain.loc[dfnoncovidtrain['patient id'] == y]
    for i, row in findValue.iterrows():
        image_value = f"{row['image name']}"
        f1 = f'/content/Data/clustercovid/coviddata/CT_NonCOVID/{image_value}'
        noncovidtrainid.append(y)
        nontrainmyFunc(f1)

def nonvalseekvalue(y):
    findValue = dfnoncovidval.loc[dfnoncovidval['patient id'] == y]
    for i, row in findValue.iterrows():
        image_value = f"{row['image name']}"
        f1 = f'/content/Data/clustercovid/coviddata/CT_NonCOVID/{image_value}'
        noncovidvalid.append(y)
        nonvalmyFunc(f1)


for i in range(len(patientidnoncovidtest)):
    x = nonseekvalue(patientidnoncovidtest[i])

for i in range(len(patientidnoncovidtrain)):
    x = nontrainseekvalue(patientidnoncovidtrain[i])

for i in range(len(patientidnoncovidval)):
    x = nonvalseekvalue(patientidnoncovidval[i])


In [ ]:
test = noncovidtest + covidtest
train = covidtrain + noncovidtrain
val = noncovidval + covidval

testlabel = covidtestlabel + noncovidtestlabel
trainlabel = noncovidtrainlabel + covidtrainlabel
vallabel = covidvallabel + noncovidvallabel

testid = noncovidtestid + covidtestid
trainid = noncovidtrainid + covidtrainid
valid = covidvalid + noncovidvalid


In [ ]:
print(np.shape(noncovidtest))
print(np.shape(noncovidtestlabel))
print(np.shape(noncovidtrain))
print(np.shape(noncovidtrainlabel))
print(np.shape(noncovidval))
print(np.shape(noncovidvallabel))

print(np.shape(noncovidtestid))
print(np.shape(noncovidtrainid))
print(np.shape(noncovidvalid))


print(np.shape(covidtest))
print(np.shape(covidtestlabel))
print(np.shape(covidtrain))
print(np.shape(covidtrainlabel))
print(np.shape(covidval))
print(np.shape(covidvallabel))

print(np.shape(covidtestid))
print(np.shape(covidtrainid))
print(np.shape(covidvalid))


(105, 200, 200, 3)
(105,)
(234, 200, 200, 3)
(234,)
(58, 200, 200, 3)
(58,)
(105,)
(234,)
(58,)
(98, 200, 200, 3)
(98,)
(191, 200, 200, 3)
(191,)
(60, 200, 200, 3)
(60,)
(98,)
(191,)
(60,)


In [ ]:
rand_order = np.random.permutation(len(test))
test = np.array(test)
test = test[rand_order, :, :, :]
testlabel = to_categorical(testlabel)
testlabel = testlabel[rand_order]
testid = np.array(testid)
testid = testid[rand_order]


rand_order = np.random.permutation(len(train))
train = np.array(train)
train = train[rand_order, :, :, :]
trainlabel = to_categorical(trainlabel)
trainlabel = trainlabel[rand_order]
trainid = np.array(trainid)
trainid = trainid[rand_order]

rand_order = np.random.permutation(len(val))
val = np.array(val)
val = val[rand_order, :, :, :]
vallabel = to_categorical(vallabel)
vallabel = vallabel[rand_order]
valid = np.array(valid)
valid = valid[rand_order]

In [ ]:
inputs = Input(shape = (desired_size, desired_size, 3))
add1 = (Conv2D(96, (3, 3), activation='relu', padding='valid'))(inputs)
max1 = (MaxPooling2D((2, 2)))(add1)
add2 = (Conv2D(256, (5, 5), activation='relu', strides=1, padding='same'))(max1)
max2 = (MaxPooling2D((3, 3)))(add2)
add3 = (Conv2D(384, (3, 3), activation='relu', strides=1, padding='same'))(max2)
add4 = (Conv2D(384, (3, 3), activation='relu', strides=1, padding='same'))(add3)
add5 = (Conv2D(256, (3, 3), activation='relu', strides=1, padding='same'))(add4)
max3 = (MaxPooling2D((3, 3)))(add5)

flattenmax = Flatten()(max3)
dense1 = (Dense(512, activation='relu'))(flattenmax)
dense2 = (Dense(512, activation='relu'))(dense1)
output = (Dense(2, activation='softmax'))(dense2)
model = Model(inputs = inputs,outputs = output)
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 200, 200, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 198, 198, 96)      2688      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 99, 99, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 99, 99, 256)       614656    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 33, 33, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 33, 33, 384)       885120    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 33, 33, 384)       132748

In [ ]:
#save weights
checkpoint_filepath = '/content/checkpoint_predictionmodel.hdf5'
model_checkpoint_callback = tf.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [ ]:
csv_log_paper = CSVLogger("prediction_paper.csv")

In [ ]:
# complie
optimizer=tf.optimizers.SGD(learning_rate=0.05)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
#history = model.fit(x=[pic1,pic2,pic3,pic4], y=endtrainlabel, batch_size=32, epochs=20,verbose=True,validation_data=([pic111,pic222,pic333,pic444],endvalabel))
history = model.fit(train, trainlabel, batch_size=32, epochs=50, callbacks=[model_checkpoint_callback],verbose=True, validation_data=(val,vallabel))

model.load_weights(checkpoint_filepath)


Epoch 1/50
14/14 [==============================] - 22s 259ms/step - loss: 0.6896 - accuracy: 0.5365 - val_loss: 0.6936 - val_accuracy: 0.4915
Epoch 2/50
14/14 [==============================] - 1s 80ms/step - loss: 0.6840 - accuracy: 0.5506 - val_loss: 0.7005 - val_accuracy: 0.4915
Epoch 3/50
14/14 [==============================] - 1s 80ms/step - loss: 0.6834 - accuracy: 0.5506 - val_loss: 0.6915 - val_accuracy: 0.4915
Epoch 4/50
14/14 [==============================] - 1s 81ms/step - loss: 0.6815 - accuracy: 0.5553 - val_loss: 0.7417 - val_accuracy: 0.4915
Epoch 5/50
14/14 [==============================] - 1s 79ms/step - loss: 0.6789 - accuracy: 0.5506 - val_loss: 0.7326 - val_accuracy: 0.4915
Epoch 6/50
14/14 [==============================] - 1s 79ms/step - loss: 0.6703 - accuracy: 0.5976 - val_loss: 0.6932 - val_accuracy: 0.4915
Epoch 7/50
14/14 [==============================] - 1s 78ms/step - loss: 0.6690 - accuracy: 0.5859 - val_loss: 0.6985 - val_accuracy: 0.5169
Epoch 8/50


In [ ]:
download_transfer = drive.CreateFile({'id':'1NTcd09yqjCh5a3kkWqiCGcclv8Nwo63q'})
download_transfer.GetContentFile('archive.zip')

In [ ]:
!unzip /content/archive.zip -d /content/transfer

Archive:  /content/archive.zip
  inflating: /content/transfer/New_Data_CoV2/Covid/Patient (10)/AnotaтАб╓╢o 2020-04-28 132320.png  
  inflating: /content/transfer/New_Data_CoV2/Covid/Patient (10)/AnotaтАб╓╢o 2020-04-28 132449.png  
  inflating: /content/transfer/New_Data_CoV2/Covid/Patient (10)/AnotaтАб╓╢o 2020-04-28 132516.png  
  inflating: /content/transfer/New_Data_CoV2/Covid/Patient (10)/AnotaтАб╓╢o 2020-04-28 132540.png  
  inflating: /content/transfer/New_Data_CoV2/Covid/Patient (10)/AnotaтАб╓╢o 2020-04-28 132608.png  
  inflating: /content/transfer/New_Data_CoV2/Covid/Patient (10)/AnotaтАб╓╢o 2020-04-28 132636.png  
  inflating: /content/transfer/New_Data_CoV2/Covid/Patient (10)/AnotaтАб╓╢o 2020-04-28 132704.png  
  inflating: /content/transfer/New_Data_CoV2/Covid/Patient (10)/AnotaтАб╓╢o 2020-04-28 132740.png  
  inflating: /content/transfer/New_Data_CoV2/Covid/Patient (10)/AnotaтАб╓╢o 2020-04-28 132814.png  
  inflating: /content/transfer/New_Data_CoV2/Covid/Patient (10)/Anota

In [ ]:
img_dir_transfer = "/content/transfer/New_Data_CoV2/Covid"
img_dirnon_transfer = "/content/transfer/New_Data_CoV2/Healthy"

healthy_patient_dirs = glob.glob("/content/transfer/New_Data_CoV2/Healthy/*/")
covid_patient_dirs = glob.glob("/content/transfer/New_Data_CoV2/Covid/*/")

# read in on an image-by-image basis
healthy_patient_imgs = glob.glob("/content/transfer/New_Data_CoV2/Healthy/*/*.png")
covid_patient_imgs = glob.glob("/content/transfer/New_Data_CoV2/Covid/*/*.png")

img_covid =[]
img_covid_label = []

img_healthy =[]
img_healthy_label = []

covidtransferid = []
healthytransferid = []

counter = 0
healthycounter = 1

for curr_dir in healthy_patient_dirs:
    imgs = glob.glob(curr_dir+'/*.png')
    for img in imgs:
        img = cv2.imread(img)
        old_size = img.shape[:2]
        ratio = float(desired_size) / max(old_size)
        new_size = tuple([int(x * ratio) for x in old_size])
        img = cv2.resize(img, (new_size[1], new_size[0]))
        delta_w = desired_size - new_size[1]
        delta_h = desired_size - new_size[0]
        top, bottom = delta_h // 2, delta_h - (delta_h // 2)
        left, right = delta_w // 2, delta_w - (delta_w // 2)
        color = [0, 0, 0]
        imgreszie = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
        normresize = (imgreszie* 1.0)/np.max(imgreszie)
        img_healthy.append(normresize)
        healthytransferid.append(healthycounter)
        img_healthy_label.append([1])
    healthycounter = healthycounter + 2

for curr_dir in covid_patient_dirs:
    imgs = glob.glob(curr_dir+'/*.png')
    for img in imgs:
        img = cv2.imread(img)
        old_size = img.shape[:2]
        ratio = float(desired_size) / max(old_size)
        new_size = tuple([int(x * ratio) for x in old_size])
        img = cv2.resize(img, (new_size[1], new_size[0]))
        delta_w = desired_size - new_size[1]
        delta_h = desired_size - new_size[0]
        top, bottom = delta_h // 2, delta_h - (delta_h // 2)
        left, right = delta_w // 2, delta_w - (delta_w // 2)
        color = [0, 0, 0]
        imgreszie = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
        normresize = (imgreszie* 1.0)/np.max(imgreszie)
        img_covid.append(normresize)
        covidtransferid.append(counter)
        img_covid_label.append([0])  
    counter = counter + 2
   



In [ ]:
print(np.shape(img_covid))
print(np.shape(covidtransferid))
print(np.shape(healthytransferid))
print(covidtransferid[0])
print(covidtransferid[1])
print(covidtransferid[2])
print(covidtransferid[3])
print(covidtransferid[4])
print(covidtransferid[5])
print(covidtransferid[6])
print(covidtransferid[7])

print(img_covid_label[0])
print(img_covid_label[1])
print(img_covid_label[2])
print(img_covid_label[3])
print(img_covid_label[4])
print(img_covid_label[5])
print(img_covid_label[6])
print(img_covid_label[7])


(2167, 200, 200, 3)
(2167,)
(757,)
0
0
0
0
0
0
0
0
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]


In [ ]:
print(healthytransferid[56])
print(img_healthy_label[56])
print(img_covid[56])


7
[1]
[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 ...

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]


In [ ]:
finaltrain = []
finaltrainlabel = []
finaltrainid = []


finalval = []
finalvallabel = []
finalvalid = []


finaltest = []
finaltestlabel = []
finaltestid = []

for i in range(0,606):
  finaltrain.append(img_healthy[i])
  finaltrainlabel.append(img_healthy_label[i])
  finaltrainid.append(healthytransferid[i])


for i in range(0,1755):
  finaltrain.append(img_covid[i])
  finaltrainlabel.append(img_covid_label[i])
  finaltrainid.append(covidtransferid[i])

for i in range(606,757):
  finaltest.append(img_healthy[i])
  finaltestlabel.append(img_healthy_label[i])
  finaltestid.append(healthytransferid[i])

for i in range(1755, 2167):
  finaltest.append(img_covid[i])
  finaltestlabel.append(img_covid_label[i])
  finaltestid.append(covidtransferid[i])

In [ ]:
print(np.shape(finaltrainid))
print(finaltrainlabel[603])
print(finaltrain[603])

(2361,)
[1]
[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 ...

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]


In [ ]:
for i in range(506,823):
  finalval.append(finaltrain[i])
  finalvallabel.append(finaltrainlabel[i])
  finalvalid.append(finaltrainid[i])

del finaltrain[506:823]
del finaltrainid[506:823]
del finaltrainlabel[506:823]

print(np.shape(finaltrainid))

(2044,)


In [ ]:
print(finaltrainlabel[0])
print(finaltrainlabel[1])

print(finaltrainlabel[2])
print(finaltrainlabel[3])
print(finaltrainlabel[4])

print(finaltrainlabel[5])
print(finaltrainlabel[6])
print(finaltrainlabel[7])
print(finaltrainlabel[8])
print(finaltrainlabel[9])


[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]


In [ ]:
print(finaltrainlabel[56])
print(finaltrainid[56])


[1]
7


In [ ]:
rand_order_train = np.random.permutation(len(finaltrain))
rand_order_test = np.random.permutation(len(finaltest))
rand_order_val = np.random.permutation(len(finalval))

finaltrain = np.array(finaltrain)
finaltrain = finaltrain[rand_order_train, :, :, :]

finaltest = np.array(finaltest)
finaltest = finaltest[rand_order_test, :, :, :]

finalval = np.array(finalval)
finalval = finalval[rand_order_val, :, :, :]

finaltrainlabel = to_categorical(finaltrainlabel)
finaltrainlabel = finaltrainlabel[rand_order_train]

finaltestlabel = to_categorical(finaltestlabel)
finaltestlabel = finaltestlabel[rand_order_test]

finalvallabel = to_categorical(finalvallabel)
finalvallabel = finalvallabel[rand_order_val]



finaltrainid = np.array(finaltrainid)
finaltrainid = finaltrainid[rand_order_train]

finaltestid = np.array(finaltestid)
finaltestid = finaltestid[rand_order_test]

finalvalid = np.array(finalvalid)
finalvalid = finalvalid[rand_order_val]

In [ ]:
print(finaltrainlabel)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [1. 0.]
 [0. 1.]]


In [ ]:
#img_data = np.array(img_data)
#img_label = to_categorical(img_label)

In [ ]:
checkpoint_filepath = "/content/checkpoint_predictionmodel.hdf5"

In [ ]:
inputs = Input(shape = (desired_size, desired_size, 3))
add1 = (Conv2D(96, (3, 3), activation='relu', padding='valid'))(inputs)
max1 = (MaxPooling2D((2, 2)))(add1)
add2 = (Conv2D(256, (5, 5), activation='relu', strides=1, padding='same'))(max1)
max2 = (MaxPooling2D((3, 3)))(add2)
add3 = (Conv2D(384, (3, 3), activation='relu', strides=1, padding='same'))(max2)
add4 = (Conv2D(384, (3, 3), activation='relu', strides=1, padding='same'))(add3)
add5 = (Conv2D(256, (3, 3), activation='relu', strides=1, padding='same'))(add4)
max3 = (MaxPooling2D((3, 3)))(add5)

flattenmax = Flatten()(max3)
dense1 = (Dense(512, activation='relu'))(flattenmax)
dense2 = (Dense(512, activation='relu'))(dense1)
output = (Dense(2, activation='softmax'))(dense2)
model = Model(inputs = inputs,outputs = output)
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 200, 200, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 198, 198, 96)      2688      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 99, 99, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 99, 99, 256)       614656    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 33, 33, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 33, 33, 384)       885120    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 33, 33, 384)       132748

In [ ]:
checkpoint_filepath_1 = '/content/please_work.hdf5'
model_checkpoint_callback = tf.callbacks.ModelCheckpoint(filepath=checkpoint_filepath_1,save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = (true_positives * 1.0) / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = (true_positives * 1.0) / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
# complie
optimizer = tf.optimizers.SGD(learning_rate=0.05)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.load_weights(checkpoint_filepath)
history = model.fit(finaltrain, finaltrainlabel, batch_size=32, epochs=50,callbacks=[model_checkpoint_callback],verbose=True,validation_data=(finalval,finalvallabel))

Epoch 1/50
64/64 [==============================] - 29s 176ms/step - loss: 0.5748 - accuracy: 0.7446 - val_loss: 0.6769 - val_accuracy: 0.6215
Epoch 2/50
64/64 [==============================] - 6s 99ms/step - loss: 0.5179 - accuracy: 0.7446 - val_loss: 0.5281 - val_accuracy: 0.6845
Epoch 3/50
64/64 [==============================] - 6s 99ms/step - loss: 0.4438 - accuracy: 0.7745 - val_loss: 0.5511 - val_accuracy: 0.7886
Epoch 4/50
64/64 [==============================] - 6s 99ms/step - loss: 0.3936 - accuracy: 0.7872 - val_loss: 0.5523 - val_accuracy: 0.6625
Epoch 5/50
64/64 [==============================] - 6s 99ms/step - loss: 0.3920 - accuracy: 0.8043 - val_loss: 0.4784 - val_accuracy: 0.7729
Epoch 6/50
64/64 [==============================] - 6s 99ms/step - loss: 0.3358 - accuracy: 0.8263 - val_loss: 0.5247 - val_accuracy: 0.7161
Epoch 7/50
64/64 [==============================] - 6s 99ms/step - loss: 0.2898 - accuracy: 0.8552 - val_loss: 0.5108 - val_accuracy: 0.7697
Epoch 8/50


In [ ]:
model.load_weights(checkpoint_filepath_1)

In [ ]:
prediction = model.predict(finaltest,verbose=1)
print(prediction)
0.42309687

18/18 [==============================] - 2s 110ms/step
[[9.9997759e-01 2.2383294e-05]
 [1.5834838e-01 8.4165162e-01]
 [9.9096090e-01 9.0390574e-03]
 ...
 [4.5500734e-01 5.4499269e-01]
 [5.9348710e-02 9.4065130e-01]
 [2.5524741e-01 7.4475253e-01]]


0.42309687

In [ ]:
def getDuplicatesWithInfo(listOfElems):
    ''' Get duplicate element in a list along with thier indices in list
     and frequency count'''
    dictOfElems = dict()
    index = 0
    # Iterate over each element in list and keep track of index
    for elem in listOfElems:
        # If element exists in dict then keep its index in lisr & increment its frequency
        if elem in dictOfElems:
            dictOfElems[elem][0] += 1
            dictOfElems[elem][1].append(index)
            
        else:
            # Add a new entry in dictionary 
            dictOfElems[elem] = [1, [index]]
        index += 1    
 
    dictOfElems = { key:value for key, value in dictOfElems.items() if value[0] > 1}
    return dictOfElems

In [ ]:
dictOfElems = getDuplicatesWithInfo(finaltestid)
for key, value in dictOfElems.items():
        print('Element = ', key , ' :: Repeated Count = ', value[0] , ' :: Index Positions =  ', value[1])  

Element =  152  :: Repeated Count =  40  :: Index Positions =   [0, 10, 14, 15, 25, 45, 64, 76, 84, 85, 97, 98, 99, 103, 175, 220, 237, 274, 303, 339, 340, 368, 373, 379, 386, 392, 398, 422, 450, 453, 456, 463, 480, 492, 493, 499, 518, 519, 526, 536]
Element =  83  :: Repeated Count =  17  :: Index Positions =   [1, 4, 13, 54, 58, 87, 225, 229, 284, 302, 360, 444, 472, 504, 515, 522, 561]
Element =  156  :: Repeated Count =  38  :: Index Positions =   [2, 17, 19, 38, 41, 47, 53, 83, 108, 110, 129, 161, 163, 171, 180, 190, 194, 197, 219, 226, 239, 245, 247, 254, 297, 299, 309, 318, 369, 375, 388, 412, 432, 440, 459, 477, 530, 558]
Element =  132  :: Repeated Count =  25  :: Index Positions =   [3, 49, 70, 72, 91, 95, 142, 156, 172, 173, 191, 298, 315, 337, 391, 396, 442, 469, 478, 488, 532, 542, 544, 550, 554]
Element =  97  :: Repeated Count =  11  :: Index Positions =   [5, 46, 138, 179, 203, 213, 291, 374, 378, 402, 405]
Element =  85  :: Repeated Count =  17  :: Index Positions =   

In [ ]:
print(finaltestlabel[59])
print(finaltestlabel[56])

[1. 0.]
[0. 1.]


In [ ]:
#compare to the label after using the threshold(giving them a proper name)
# diagrams, table with f1, acc, per, confusion for the final model, if time(misclass images and classified images and roc curve), 
import tensorflow as tf

finalpred = []
actuallabel = []

for key, value in dictOfElems.items():
       sum = 0
       evaluation =0
       for i in range(dictOfElems[key][0]):
         print(prediction[dictOfElems[key][1][i]][1])
         sum = sum + prediction[dictOfElems[key][1][i]][1]
       print(dictOfElems[key][0])
       sum = sum/dictOfElems[key][0]
       print(sum)
       if(sum < .50):
         evaluation = 0
         finalpred.append(tf.cast(evaluation,tf.float32))
       elif(sum > .50):
         evaluation = 1
         finalpred.append(tf.cast(evaluation,tf.float32))
       actualvalue = finaltestlabel[dictOfElems[key][1][i]]
       actuallabel.append(tf.cast(actualvalue[1],tf.float32))
       print('Element = ', key ,' :: prediction evaluation = ',evaluation,' :: label = ',actualvalue[1] )
finalpred = np.array(finalpred)
actuallabel = np.array(actuallabel)

2.2383294e-05
2.7086373e-05
2.3204632e-06
1.5928947e-05
3.343363e-05
7.1559946e-08
0.00013610873
1.5336487e-06
6.634884e-07
0.00011525956
8.4727735e-06
0.00019259332
2.4948613e-07
0.01387433
5.027252e-05
8.6908706e-08
3.172874e-06
7.441703e-07
0.005292106
0.00024890387
0.0014426138
0.00019835624
1.1204849e-05
0.000918055
1.0059148e-06
1.0779283e-05
0.008110174
2.4180403e-05
0.00016680446
7.139756e-07
8.355522e-05
2.9884168e-05
0.0006007076
6.822974e-06
4.423241e-05
0.0028615694
4.9544014e-05
0.00040484068
0.0001415213
2.4169613e-05
40
0.0008789114120013152
Element =  152  :: prediction evaluation =  0  :: label =  0.0
0.8416516
0.80420387
0.95888305
0.6935397
0.26098493
0.29353034
0.07817027
0.7140486
0.9175207
0.8647692
0.9006594
0.6744492
0.31370553
0.5345495
0.9620461
0.52655894
0.9406513
17
0.6635248363018036
Element =  83  :: prediction evaluation =  1  :: label =  1.0
0.009039057
1.3003082e-06
3.0545657e-09
0.0016793085
0.23647185
0.0011366974
0.17342596
0.4035937
0.024254298
0.0

In [ ]:
print(actuallabel)

[0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 1.]


In [ ]:
print(finalpred)

[0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0.
 0.]


In [ ]:
recallvalue = recall_m(actuallabel,finalpred)
print(f'Test recall: {recallvalue:.3}')

In [ ]:
precvalue = precision_m(actuallabel,finalpred)
print(f'Test precision: {precvalue:.3}')

In [ ]:
f1value = f1_m(actuallabel,finalpred)
print(f'Test f1_score: {f1value:.3}')

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import itertools
import matplotlib.pyplot as plt





print(classification_report(y_true=actuallabel, y_pred=finalpred))

tn, fp, fn, tp = confusion_matrix(y_true=actuallabel, y_pred=finalpred).ravel()
(tn, fp, fn, tp)

              precision    recall  f1-score   support

         0.0       0.88      1.00      0.94        15
         1.0       1.00      0.80      0.89        10

    accuracy                           0.92        25
   macro avg       0.94      0.90      0.91        25
weighted avg       0.93      0.92      0.92        25



(15, 0, 2, 8)